In [1]:
import tensorflow as tf
import os
import keras.backend as K
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model
from dataset import *

task_names = ['skirt_length', 'coat_length', 'collar_design', 'neck_design', 'neckline_design', 'pant_length', 'sleeve_length', 'lapel_design']



Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

In [3]:
task_name = task_names[4]
df = pd.read_csv('data/Annotations/label.csv', header=None)
df.columns = ['filename', 'label_name', 'label']

df.label_name = df.label_name.str.replace('_labels', '')
df = df[df.label_name == task_name]
df = df.sample(frac=1).reset_index(drop=True) # shuffle

c = Counter(df.label_name)
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
n = len(df)
fnames = df['filename'].values
width = 399

y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
df = df.reset_index(drop=True)

def f(index):
    if os.path.isfile('data/'+fnames[index]):
        return index, cv2.resize(cv2.imread('data/'+fnames[index]), (width, width))

for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1
    #y[label_names.index(label_name)][i, label.find('m')] = 0.5

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

n_train = int(n*0.8)

X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

gen_train = Generator(X_train, y_train, batch_size=16, aug=True)

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
base_model = InceptionV3(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling = 'avg')

input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

gpu_model = multi_gpu_model(model, n_gpus)


100%|██████████| 17148/17148 [00:39<00:00, 430.76it/s]


In [4]:
gpu_model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
gpu_model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/3
858/858 [==============================] - 698s 814ms/step - loss: 1.2482 - acc: 0.5816 - val_loss: 0.7783 - val_acc: 0.7475
Epoch 2/3
858/858 [==============================] - 611s 712ms/step - loss: 0.7366 - acc: 0.7661 - val_loss: 0.6949 - val_acc: 0.7883
Epoch 3/3
858/858 [==============================] - 611s 712ms/step - loss: 0.5706 - acc: 0.8199 - val_loss: 0.5676 - val_acc: 0.8303


In [5]:
gpu_model.compile(optimizer=Adam(0.000025), loss='categorical_crossentropy', metrics=['accuracy'])
gpu_model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=2, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/2
858/858 [==============================] - 648s 755ms/step - loss: 0.3736 - acc: 0.8811 - val_loss: 0.4679 - val_acc: 0.8513
Epoch 2/2
858/858 [==============================] - 569s 663ms/step - loss: 0.3058 - acc: 0.9029 - val_loss: 0.4853 - val_acc: 0.8493


In [6]:
gpu_model.compile(optimizer=Adam(0.00000625), loss='categorical_crossentropy', metrics=['accuracy'])
gpu_model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/3
858/858 [==============================] - 655s 764ms/step - loss: 0.2450 - acc: 0.9232 - val_loss: 0.4564 - val_acc: 0.8580
Epoch 2/3
858/858 [==============================] - 568s 661ms/step - loss: 0.2244 - acc: 0.9291 - val_loss: 0.4666 - val_acc: 0.8609
Epoch 3/3
858/858 [==============================] - 568s 662ms/step - loss: 0.2017 - acc: 0.9362 - val_loss: 0.4693 - val_acc: 0.8609


In [7]:
gpu_model.compile(optimizer=Adam(0.00000425), loss='categorical_crossentropy', metrics=['accuracy'])
gpu_model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/1
858/858 [==============================] - 675s 787ms/step - loss: 0.1912 - acc: 0.9406 - val_loss: 0.4655 - val_acc: 0.8624


In [8]:
gpu_model.compile(optimizer=Adam(0.000001), loss='categorical_crossentropy', metrics=['accuracy'])
gpu_model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)


Epoch 1/1
858/858 [==============================] - 686s 799ms/step - loss: 0.1835 - acc: 0.9425 - val_loss: 0.4662 - val_acc: 0.8603


In [9]:
gpu_model.compile(optimizer=SGD(lr=1e-8, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
gpu_model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)


Epoch 1/1
858/858 [==============================] - 636s 741ms/step - loss: 0.1722 - acc: 0.9457 - val_loss: 0.4655 - val_acc: 0.8592


In [11]:
# for c in range(8):
y_pred = model.predict(X_valid, batch_size=128, verbose=1)
y_pred2 = np.array(y_pred).argmax(axis=-1)
y_true2 = np.array(y_valid).argmax(axis=-1)
print(task_name, (y_pred2 == y_true2).mean())



3430/3430 [==============================] - 101s 29ms/step
neckline_design 0.8591836734693877
